In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from datasets import load_dataset
import plotting
from util import tokenize_tulu_dataset, get_checkpoints_olmo
import pandas as pd
import torch
from transformers import AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-2-1124-7B")

In [3]:
import plotting

In [4]:
checkpoints = pd.DataFrame(get_checkpoints_olmo())
checkpoints

,0
0,stage1-step150-tokens1B
1,stage1-step600-tokens3B
2,stage1-step700-tokens3B
3,stage1-step850-tokens4B
4,stage1-step900-tokens4B
...,...
960,stage2-ingredient3-step9000-tokens38B
961,stage2-ingredient3-step10000-tokens42B
962,stage2-ingredient3-step11000-tokens47B
963,stage2-ingredient3-step11931-tokens50B


In [5]:
dataset_name = "allenai/tulu-v2-sft-mixture"
model_name = "allenai/OLMo-2-1124-7B"

dataset = tokenize_tulu_dataset("allenai/tulu-v2-sft-mixture")
influence_output_dir = os.path.join("./influence", os.path.basename(model_name))

Parameter 'function'=functools.partial(<function preprocess_tulu at 0x7f577502a980>, tokenizer=GPT2TokenizerFast(name_or_path='allenai/OLMo-2-1124-7B', vocab_size=100278, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|pad|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	100256: AddedToken("<|extra_id_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	100257: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100258: AddedToken("<|fim_prefix|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100259: AddedToken("<|fim_middle|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100260: AddedToken("<|fim_suffix|>", rstrip=False, lstr

Map (num_proc=20):   0%|          | 0/3262 [00:00<?, ? examples/s]

Filter (num_proc=20):   0%|          | 0/3262 [00:00<?, ? examples/s]

In [6]:
os.listdir(influence_output_dir)

['main', 'tokens1B']

In [7]:
df = pd.DataFrame({(result_checkpoint): torch.load(os.path.join(influence_output_dir,result_checkpoint),weights_only=True,map_location="cpu").numpy().flatten() for result_checkpoint in os.listdir(influence_output_dir)})
df.sort_index(axis=1)

df = df.reindex(sorted(df.columns, reverse=False), axis=1)
influence_cols = df.columns
df["total"] = df.sum(axis=1)
df["text"] = dataset.to_pandas()["input_ids"].apply(lambda x: tokenizer.decode(x, skip_special_tokens=True))
# df["document_lenght"] = df["text"].str.split().str.len()

In [8]:
pd.set_option('display.max_colwidth', 1000)
df.sort_values(by="total", ascending=False)

,main,tokens1B,total,text
1106,0.262416,1.767627,2.030043,<|user|>\nProblem: Write a brief sentence.\nAnswer: Two sisters contemplate jumping in the lake.\nProblem: Write a brief sentence.\nAnswer: The woman is selling the vegetables.\nProblem: Write a brief sentence.\nAnswer:\n<|assistant|>\nA priest gives a sermon from the pulpit.\n
1758,0.236128,1.792152,2.028280,<|user|>\nProblem: Write a brief sentence.\nAnswer: A young male walks through the desert.\nProblem: Write a brief sentence.\nAnswer: A woman is riding a bicycle.\nProblem: Write a brief sentence.\nAnswer:\n<|assistant|>\nThe mimes are amused.\n
2861,0.212293,1.800123,2.012416,<|user|>\nProblem: Write a brief sentence.\nAnswer: The boy is wearing red pants\nProblem: Write a brief sentence.\nAnswer: The man is in a car.\nProblem: Write a brief sentence.\nAnswer:\n<|assistant|>\nA boy wearing a black helmet is skateboarding.\n
1782,0.200797,1.786634,1.987431,<|user|>\nProblem: Write a brief sentence.\nAnswer: There is an animal hunting.\nProblem: Write a brief sentence.\nAnswer: The man is giving a bike tour of buildings in a city.\nProblem: Write a brief sentence.\nAnswer:\n<|assistant|>\nThe shirt is dark.\n
2901,0.199264,1.777437,1.976701,<|user|>\nProblem: Write a brief sentence.\nAnswer: A man is performing on stage\nProblem: Write a brief sentence.\nAnswer: No one is standing.\nProblem: Write a brief sentence.\nAnswer:\n<|assistant|>\nThe dog jumps into the water to retrieve the ball for his mistress\n
...,...,...,...,...
2918,0.013125,0.020214,0.033338,"<|user|>\nExperts warn luxury Arctic cruises of Titanic-style disaster\n\nExperts are warning there could be a Titanic-style disaster and serious environmental damage unless controls are brought in for cruise ships in the Arctic Ocean.\n\nThe first luxury ship sailed through the Arctic's remote Northwest Passage this summer, prompting fears more could follow suit before safety concerns have been addressed.\n\nSome 1,700 passengers paid a minimum of $19,755 for a berth on the Crystal Serenity, which left Anchorage in Alaska on 15 August.\n\nIt cruised through the Passage along the northern coast of North America, before docking in New York a month later. There is no port on the journey between Alaska and Greenland.\n\nThe ship, owned by American operator Crystal Cruises, traversed an isolated route first navigated by Norwegian explorer Roald Amundsen in 1903.\n\nAlthough climate change means the region is iceberg free in summer, two shipping executives fear a disaster unless guideli..."
306,0.003901,0.026192,0.030093,"<|user|>\nProblem: Eating and drinking too much sugar can be bad for your health, but sometimes it's tricky to know exactly how much sugar is in the food we buy.\nSo now an app supported by the government in England, has been launched to help you work out what's in your food. You scan the item's barcode and it should tell you the amount of sugar in it.\nRicky went to meet some kids who have been keeping a close eye on what they eat.\nWhat was that article about?\n\nA: An app supported by the government is designed to make it easier to stay healthy, by letting you know exactly what is in your food.\n\n\n""The nation"" is a flexible concept here, as this is a Scottish viewpoint, when everyone agrees that the energy market should remain British, if not expanding into a European single market, with the help of more sub-sea inter-connectors.\nAppearing before the enterprise and energy committee are the big energy utilities, and their regulator. Scottish Power and SSE (Scottish Hydro, to ..."
493,0.010960,0.016880,0.027840,"<|user|>\n– Iconic funnywoman Phyllis Diller has died in her Los Angeles home at the age of 95, reports TMZ. The trailblazing comedienne who rose to fame alongside Bob Hope had been in hospice care following a recent fall in which she injured her wrist and hip. “Phyllis Diller passed away peacefully in her sleep at 9:30am PT,"" her manager tells CNN. ""Her son, Perry, found her with a

In [9]:
tokenizer.decode(dataset[1000]["input_ids"], skip_special_tokens=False)

'<|endoftext|><|user|>\nQuestion:\nWrite a sentence not in English.\nSentence:\nLa tradition orale autochtone évoque les corps rejetés dans la baie Dry, sur la côte de l’Alaska.\n-+-+-+-\nQuestion:\nWrite a sentence not in English.\nSentence:\nDans une étude de la saison de pêche 2000 effectuée par le Groupe de travail technique sur les pêches, le MPO et la Nation Nisga’a ont déterminé les changements qui devaient être apportés aux critères d’allocation de la ressource.\n-+-+-+-\nQuestion:\nWrite a sentence not in English.\nSentence:\nLE PRÉSIDENT (Interprétation) constate l’absence de M. Dedja et de M. Coifan et donne la parole à M. Ékes.\n-+-+-+-\nQuestion:\nWrite a sentence not in English.\nSentence:\n<|assistant|>\nLe mandat premier du Centre est d’assurer des services d’aide juridique aux requérants selon le nouveau processus de traitement des requêtes.\n-+-+-+-<|endoftext|>\n<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><

In [10]:
dataset[0]["input_ids"]

[100257,
 27,
 91,
 882,
 91,
 397,
 14924,
 25,
 480,
 3315,
 19699,
 4991,
 320,
 11,
 2652,
 6063,
 25,
 1621,
 1174,
 883,
 374,
 264,
 33084,
 3363,
 389,
 279,
 73089,
 13962,
 13,
 1102,
 374,
 279,
 6864,
 315,
 279,
 393,
 26429,
 75612,
 29179,
 344,
 2601,
 2200,
 11,
 28702,
 596,
 12717,
 513,
 391,
 371,
 323,
 374,
 1101,
 279,
 12541,
 315,
 279,
 3224,
 596,
 11999,
 68067,
 58506,
 3158,
 13,
 4815,
 791,
 3363,
 15812,
 389,
 279,
 18561,
 6964,
 315,
 480,
 3315,
 19699,
 4991,
 9332,
 320,
 1073,
 279,
 73089,
 15379,
 705,
 304,
 264,
 390,
 65916,
 449,
 279,
 3363,
 315,
 480,
 44185,
 689,
 11,
 31493,
 6424,
 315,
 328,
 89490,
 11,
 323,
 46318,
 10977,
 11,
 902,
 3871,
 1376,
 264,
 58506,
 3158,
 2663,
 279,
 1183,
 62791,
 3573,
 1305,
 62424,
 8318,
 54485,
 4063,
 449,
 264,
 7187,
 31047,
 220,
 16,
 13,
 19,
 3610,
 13,
 480,
 3315,
 19699,
 4991,
 5196,
 706,
 264,
 7187,
 315,
 220,
 16551,
 11,
 20465,
 320,
 33246,
 220,
 679,
 17,
 705,
 3339,
 4

In [11]:
dataset[0]

{'messages': [{'role': 'user',
   'content': 'Question: Gdańsk (, ; German: "" , ) is a Polish city on the Baltic coast. It is the capital of the Pomeranian Voivodeship, Poland\'s principal seaport and is also the centre of the country\'s fourth-largest metropolitan area. \n\nThe city lies on the southern edge of Gdańsk Bay (of the Baltic Sea), in a conurbation with the city of Gdynia, spa town of Sopot, and suburban communities, which together form a metropolitan area called the Tricity ("Trójmiasto"), with a population approaching 1.4 million. Gdańsk itself has a population of 460,427 (December 2012), making it the largest city in the Pomerania region of Northern Poland. \n\nGdańsk is the capital of Gdańsk Pomerania and the largest city of Kashubia. With its origins as a Polish stronghold erected in the 980s by Mieszko I of Poland, the city\'s history is complex, with periods of Polish rule, periods of Prussian or German rule, and periods of autonomy or self-rule as a "free city". Be

In [12]:
from pathlib import Path
import numpy as np





In [13]:
checkpoint_names

NameError: name 'checkpoint_names' is not defined

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch




In [ ]:

message = ["Language modeling is "]

oldmos = []
for i in [1,100,500,900]:
    
    olmo = AutoModelForCausalLM.from_pretrained("allenai/OLMo-2-1124-7B", revision=checkpoint_names[i])
    inputs = tokenizer(message, return_tensors='pt', return_token_type_ids=False)
    # optional verifying cuda
    inputs = {k: v.to('cuda') for k,v in inputs.items()}
    olmo = olmo.to('cuda')
    oldmos.append(olmo)
    response = olmo.generate(**inputs, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)
    print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

model-00001-of-00006.safetensors:   7%|6         | 336M/4.95G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

...

model-00002-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.61G [00:00<?, ?B/s]

In [8]:

checkpoints_stage_2

[['stage2-ingredient1-step1000-tokens5B',
  'stage2',
  'ingredient1',
  'step1000',
  'tokens5B'],
 ['stage2-ingredient1-step2000-tokens9B',
  'stage2',
  'ingredient1',
  'step2000',
  'tokens9B'],
 ['stage2-ingredient1-step3000-tokens13B',
  'stage2',
  'ingredient1',
  'step3000',
  'tokens13B'],
 ['stage2-ingredient1-step4000-tokens17B',
  'stage2',
  'ingredient1',
  'step4000',
  'tokens17B'],
 ['stage2-ingredient1-step5000-tokens21B',
  'stage2',
  'ingredient1',
  'step5000',
  'tokens21B'],
 ['stage2-ingredient1-step6000-tokens26B',
  'stage2',
  'ingredient1',
  'step6000',
  'tokens26B'],
 ['stage2-ingredient1-step7000-tokens30B',
  'stage2',
  'ingredient1',
  'step7000',
  'tokens30B'],
 ['stage2-ingredient1-step8000-tokens34B',
  'stage2',
  'ingredient1',
  'step8000',
  'tokens34B'],
 ['stage2-ingredient1-step9000-tokens38B',
  'stage2',
  'ingredient1',
  'step9000',
  'tokens38B'],
 ['stage2-ingredient1-step10000-tokens42B',
  'stage2',
  'ingredient1',
  'step10000'

In [ ]:
checkpoints_stage1

NameError: name 'checkpoints_stage1' is not defined

[['stage1-step237000-tokens995B', 'stage1', 'step237000', 'tokens995B'],
 ['stage1-step236000-tokens990B', 'stage1', 'step236000', 'tokens990B'],
 ['stage1-step235000-tokens986B', 'stage1', 'step235000', 'tokens986B'],
 ['stage1-step234000-tokens982B', 'stage1', 'step234000', 'tokens982B'],
 ['stage1-step233000-tokens978B', 'stage1', 'step233000', 'tokens978B'],
 ['stage1-step232000-tokens974B', 'stage1', 'step232000', 'tokens974B'],
 ['stage1-step231000-tokens969B', 'stage1', 'step231000', 'tokens969B'],
 ['stage1-step230000-tokens965B', 'stage1', 'step230000', 'tokens965B'],
 ['stage1-step229000-tokens961B', 'stage1', 'step229000', 'tokens961B'],
 ['stage1-step228000-tokens957B', 'stage1', 'step228000', 'tokens957B'],
 ['stage1-step227000-tokens953B', 'stage1', 'step227000', 'tokens953B'],
 ['stage1-step226000-tokens948B', 'stage1', 'step226000', 'tokens948B'],
 ['stage1-step225000-tokens944B', 'stage1', 'step225000', 'tokens944B'],
 ['stage1-step224000-tokens940B', 'stage1', 'step22